In [4]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


ModuleNotFoundError: No module named 'tensorflow'

In [6]:
# --- IMPORTS ---
import os
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# --- PATH SETUP ---
BASE_DIR = os.getcwd()
DATASET_PATH = os.path.join(BASE_DIR, "dataset")

TRAIN_DIR = os.path.join(DATASET_PATH, "train")
VAL_SPLIT = 0.2

# --- DATA PREPROCESSING ---
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    validation_split=VAL_SPLIT
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(224, 224),
    batch_size=16,   # smaller batch for less memory use
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)

# --- MODEL BUILDING ---
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze all base layers

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# --- TRAINING ---
EPOCHS = 5  # increase later if needed
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    verbose=1
)

# --- SAVE MODEL & WEIGHTS ---
model_save_dir = os.path.join(BASE_DIR, "saved_models")
os.makedirs(model_save_dir, exist_ok=True)

model.save(os.path.join(model_save_dir, "best_vgg_finetuned_smallrun.h5"))
model.save_weights(os.path.join(model_save_dir, "model_weights.weights.h5"))


# Save architecture JSON
model_json = model.to_json()
with open(os.path.join(model_save_dir, "model_architecture.json"), "w") as json_file:
    json_file.write(model_json)

print("✅ Training complete — model and architecture saved in /saved_models/")


ModuleNotFoundError: No module named 'tensorflow'